# [ 5-2. 추정결과 출력 파일 작성 ]

## 1. 기본 모듈 insert

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.width', 300)

# DataFrame의 출력을 확장하여 한 줄로 계속 출력되도록 설정
pd.set_option('display.expand_frame_repr', True)

from m00_general_function import df_seperator
from m01_assumption import assumption
from m02_index import index
from m03_funding import funding
from m04_operating_income import operating_income
from m05_operating_cost import operating_cost
from m06_facility_cost import facility_cost
from m07_cashflow import cashflow, balance

## 2. 피벗테이블 return 함수 작성

In [2]:
def pivot_cashflow(cashflow=cashflow):
    cf_table = cashflow.copy()
    
    # 입금금액과 출금금액 간 차액 계산
    cf_table['amount'] = cf_table['입금금액'] - cf_table['출금금액']
    cf_table = cf_table[['date', 'categoryA', 'categoryB', 'categoryC', 'amount']]

    # 피벗테이블 작성
    cf_table = cf_table.pivot_table(
        values = 'amount',
        index = 'date',
        columns = ['categoryA', 'categoryB', 'categoryC'],
        aggfunc = 'sum'
    ).fillna(0).astype(int)

    # 현금잔액 및 소계 데이터 추가
    cf_table[('운영수입', '소계', '')] = operating_income['Total']['객실수입']
    cf_table[('객실운영비', '소계', '')] = -operating_cost['객실운영비']['Total']
    cf_table[('관리운영비', '소계', '')] = -operating_cost['관리운영비']['Total']
    cf_table[('인건비', '소계', '')] = -operating_cost['인건비']['Total']
    cf_table[('시설관리비', '소계', '')] = -facility_cost['Total']
    cf_table['기초현금'] = balance['기초현금']
    cf_table['기말현금'] = balance['기말현금']

    # 컬럼 순서 정리
    cf_table = cf_table[[
        (   '기초현금',       '',         ''),
        (   '자금조달',   '자기자본',   '자기자본유입'),
        (   '자금조달',    '차입금',    '차입금유입'),
        (   '자산매입',   '자산매입',  '매입대금지출'),
        (   '자산매입',   '매입부수비용',  '부수비용지출'),
        (   '운영수입',   '객실수입',     'TypeA'),
        (   '운영수입',   '객실수입',     'TypeB'),
        (   '운영수입',   '객실수입',     'TypeC'),
        (   '운영수입',      '소계',          ''),
        (  '객실운영비',  '예약수수료',    'TypeA'),
        (  '객실운영비',  '예약수수료',    'TypeB'),
        (  '객실운영비',  '예약수수료',    'TypeC'),
        (  '객실운영비',  '청소세탁비',    'TypeA'),
        (  '객실운영비',  '청소세탁비',    'TypeB'),
        (  '객실운영비',  '청소세탁비',    'TypeC'),
        (  '객실운영비',  '수도광열비',    'TypeA'),
        (  '객실운영비',  '수도광열비',    'TypeB'),
        (  '객실운영비',  '수도광열비',    'TypeC'),
        (  '객실운영비',  '수도광열비', 'Overhead'),
        (  '객실운영비',      '소계',         ''),
        (  '관리운영비',  '관리운영비',   '광고홍보비'),
        (  '관리운영비',  '관리운영비',   '기타운영비'),
        (  '관리운영비',      '소계',         ''),
        (    '인건비',  '객실운영팀',      '임시직'),
        (    '인건비',  '객실운영팀',      '정규직'),
        (    '인건비',  '경영지원팀',       '임원'),
        (    '인건비',  '경영지원팀',      '정규직'),
        (    '인건비',   '마케팅팀',      '정규직'),
        (    '인건비',  '시설관리팀',      '임시직'),
        (    '인건비',  '시설관리팀',      '정규직'),
        (    '인건비',      '소계',         ''),
        (  '시설관리비',  '통상수선비',    'TypeA'),
        (  '시설관리비',  '통상수선비',    'TypeB'),
        (  '시설관리비',  '통상수선비',    'TypeC'),
        (  '시설관리비', '대수선공사비',    'TypeA'),
        (  '시설관리비', '대수선공사비',    'TypeB'),
        (  '시설관리비', '대수선공사비',    'TypeC'),
        (  '시설관리비',      '소계',         ''),
        ('차입원리금상환',    '차입금',    '차입금상환'),
        ('차입원리금상환',    '차입금',     '차입이자'),     
        ('자기자본배당', '자기자본배당', '배당금지급'),   
        (   '기말현금',       '',         '')
    ]]

    return cf_table

In [3]:
pivot_cashflow(cashflow)

categoryA         기초현금         자금조달                      자산매입                   운영수입                                     객실운영비                                                                                                   관리운영비                          인건비                                        \
categoryB                      자기자본          차입금         자산매입      매입부수비용       객실수입                               소계    예약수수료                       청소세탁비                        수도광열비                                   소계     관리운영비                  소계    객실운영팀              경영지원팀               마케팅팀   
categoryC                    자기자본유입        차입금유입       매입대금지출      부수비용지출      TypeA      TypeB      TypeC               TypeA    TypeB    TypeC     TypeA     TypeB     TypeC    TypeA    TypeB    TypeC Overhead               광고홍보비     기타운영비                임시직       정규직       임원       정규직      정규직   
date                                                                                                                                                                                                                                                                                                        
2023-12-31           0  10000000000  10000000000 -18000000000 -1000000000          0          0          0          0        0        0        0         0         0         0        0        0        0 -3000000  -3000000 -30000000 -10000000 -40000000 -5095890 -15287671 -6794520  -9172602 -6115068   
2024-01-31   905871236            0            0            0           0  145080000  167400000  131750000  444230000 -4352400 -5022000 -3952500 -11160000 -13392000 -10540000 -5580000 -6696000 -5270000 -3000000 -68964900 -30000000 -10000000 -40000000 -5081967 -15245901 -6775956  -9147540 -6098360   
2024-02-29  1144288339            0            0            0           0  128180000  147900000  116000000  392080000 -3845400 -4437000 -3480000  -9860000 -11832000  -9280000 -4930000 -5916000 -4640000 -3000000 -61220400 -30000000 -10000000 -40000000 -4754098 -14262295 -6338797  -8557377 -5704918   
2024-03-31  1344733797            0            0            0           0   99200000  119040000   86800000  305040000 -2976000 -3571200 -2604000  -9920000 -11904000  -8680000 -4960000 -5952000 -4340000 -3000000 -57907200 -10000000 -10000000 -20000000 -5081967 -15245901 -6775956  -9147540 -6098360   
2024-04-30  1425477400            0            0            0           0   96000000  115200000   84000000  295200000 -2880000 -3456000 -2520000  -9600000 -11520000  -8400000 -4800000 -5760000 -4200000 -3000000 -56136000 -10000000 -10000000 -20000000 -4918032 -14754098 -6557377  -8852459 -5901639   
...                ...          ...          ...          ...         ...        ...        ...        ...        ...      ...      ...      ...       ...       ...       ...      ...      ...      ...      ...       ...       ...       ...       ...      ...       ...      ...       ...      ...   
2026-09-30  1747490404            0            0            0           0   96000000  115200000   90000000  301200000 -2880000 -3456000 -2700000 -10184640 -12221568  -9548100 -5092320 -6110784 -4774050 -3182700 -60150162 -10609000 -10609000 -21218000 -5436986 -16310958 -7249315  -9786575 -6524383   
2026-10-31  1820534619            0            0            0           0   99200000  119040000   93000000  311240000 -2976000 -3571200 -2790000 -10524128 -12628953  -9866370 -5262064 -6314476 -4933185 -3182700 -62049076 -10609000 -10609000 -21218000 -5618219 -16854657 -7490958 -10112794 -6741863   
2026-11-30  1948705996            0            0            0           0   84000000  100800000   84000000  268800000 -2520000 -3024000 -2520000  -8911560 -10693872  -8911560 -4455780 -5346936 -4455780 -3182700 -54022188 -10609000 -10609000 -21218000 -5436986 -16310958 -7249315  -9786575 -6524383   
2026-12-31  2046027396            0            0            0           0  145080000  1

## 3. 엑셀파일 생성 함수 작성

In [4]:
def print_cashflow(cashflow=cashflow, file_name='cashflow_table.xlsx'):
    cf_excel = pivot_cashflow(cashflow)
    cf_excel.index = cf_excel.index.strftime('%Y-%m-%d')
    cf_excel.to_excel(file_name)

In [5]:
print_cashflow(cashflow, 'cashflow_table_test.xlsx')

## 4. cashflow_results.py 파일 작성
### 4-1. 코드 내용을 새로 생성한 파일에 복사
* cashflow_results.py 파일 생성 후 코드 내용 복사

### 4-2. 객체 종합 dictionary 작성

In [6]:
dct = {
    'assumption': assumption,
    'index': index,
    'funding': funding,
    'operating_income': operating_income,
    'operating_cost': operating_cost,
    'facility_cost': facility_cost,
    'cashflow': cashflow,
    'balance': balance,
    'pivot_cashflow': pivot_cashflow,
    'print_cashflow': print_cashflow
}

## 5. cashflow_results.py 파일 사용
* "재시작" 클릭 후 아래 내용 진행

In [7]:
import m08_cashflow_results as cf

In [8]:
cf.pivot_cashflow()

categoryA         기초현금         자금조달                      자산매입                   운영수입                                     객실운영비                                                                                                   관리운영비                          인건비                                        \
categoryB                      자기자본          차입금         자산매입      매입부수비용       객실수입                               소계    예약수수료                       청소세탁비                        수도광열비                                   소계     관리운영비                  소계    객실운영팀              경영지원팀               마케팅팀   
categoryC                    자기자본유입        차입금유입       매입대금지출      부수비용지출      TypeA      TypeB      TypeC               TypeA    TypeB    TypeC     TypeA     TypeB     TypeC    TypeA    TypeB    TypeC Overhead               광고홍보비     기타운영비                임시직       정규직       임원       정규직      정규직   
date                                                                                                                                                                                                                                                                                                        
2023-12-31           0  10000000000  10000000000 -18000000000 -1000000000          0          0          0          0        0        0        0         0         0         0        0        0        0 -3000000  -3000000 -30000000 -10000000 -40000000 -5095890 -15287671 -6794520  -9172602 -6115068   
2024-01-31   905871236            0            0            0           0  145080000  167400000  131750000  444230000 -4352400 -5022000 -3952500 -11160000 -13392000 -10540000 -5580000 -6696000 -5270000 -3000000 -68964900 -30000000 -10000000 -40000000 -5081967 -15245901 -6775956  -9147540 -6098360   
2024-02-29  1144288339            0            0            0           0  128180000  147900000  116000000  392080000 -3845400 -4437000 -3480000  -9860000 -11832000  -9280000 -4930000 -5916000 -4640000 -3000000 -61220400 -30000000 -10000000 -40000000 -4754098 -14262295 -6338797  -8557377 -5704918   
2024-03-31  1344733797            0            0            0           0   99200000  119040000   86800000  305040000 -2976000 -3571200 -2604000  -9920000 -11904000  -8680000 -4960000 -5952000 -4340000 -3000000 -57907200 -10000000 -10000000 -20000000 -5081967 -15245901 -6775956  -9147540 -6098360   
2024-04-30  1425477400            0            0            0           0   96000000  115200000   84000000  295200000 -2880000 -3456000 -2520000  -9600000 -11520000  -8400000 -4800000 -5760000 -4200000 -3000000 -56136000 -10000000 -10000000 -20000000 -4918032 -14754098 -6557377  -8852459 -5901639   
...                ...          ...          ...          ...         ...        ...        ...        ...        ...      ...      ...      ...       ...       ...       ...      ...      ...      ...      ...       ...       ...       ...       ...      ...       ...      ...       ...      ...   
2026-09-30  1747490404            0            0            0           0   96000000  115200000   90000000  301200000 -2880000 -3456000 -2700000 -10184640 -12221568  -9548100 -5092320 -6110784 -4774050 -3182700 -60150162 -10609000 -10609000 -21218000 -5436986 -16310958 -7249315  -9786575 -6524383   
2026-10-31  1820534619            0            0            0           0   99200000  119040000   93000000  311240000 -2976000 -3571200 -2790000 -10524128 -12628953  -9866370 -5262064 -6314476 -4933185 -3182700 -62049076 -10609000 -10609000 -21218000 -5618219 -16854657 -7490958 -10112794 -6741863   
2026-11-30  1948705996            0            0            0           0   84000000  100800000   84000000  268800000 -2520000 -3024000 -2520000  -8911560 -10693872  -8911560 -4455780 -5346936 -4455780 -3182700 -54022188 -10609000 -10609000 -21218000 -5436986 -16310958 -7249315  -9786575 -6524383   
2026-12-31  2046027396            0            0            0           0  145080000  1

In [9]:
cf.cashflow

,date,categoryA,categoryB,categoryC,입금금액,출금금액
0,2023-12-31,자금조달,자기자본,자기자본유입,10000000000,0
1,2023-12-31,자금조달,차입금,차입금유입,10000000000,0
2,2023-12-31,자산매입,자산매입,매입대금지출,0,18000000000
3,2023-12-31,자산매입,매입부수비용,부수비용지출,0,1000000000
4,2023-12-31,객실운영비,수도광열비,Overhead,0,3000000
...,...,...,...,...,...,...
957,2027-01-31,인건비,경영지원팀,정규직,0,10618434
958,2027-01-31,인건비,경영지원팀,임원,0,7865506
959,2027-01-31,인건비,마케팅팀,정규직,0,7078956
960,2027-01-31,인건비,시설관리팀,정규직,0,7078956


In [10]:
cf.dct

{'assumption': {'business_overview': {'사업개요': {'사업내용': '호텔건물 매입 및 운영 사업',
    '건물명': 'FS호텔',
    '주소': '서울시 강남구 테헤란로',
    '건물규모': '지하1층/지상10층'},
   '객실수': {'TypeA': 40, 'TypeB': 40, 'TypeC': 20},
   '자산매입': {'자산매입금액': 18000000000, '매입부수비용': 1000000000}},
  'period_assumptions': {'기본기간가정': {'모델시작일': datetime.datetime(2023, 12, 1, 0, 0),
    '모델종료일': datetime.datetime(2027, 1, 31, 0, 0),
    '운영시작일': datetime.datetime(2024, 1, 1, 0, 0),
    '운영종료일': datetime.datetime(2026, 12, 31, 0, 0)},
   '자산매입일정': {'자산매입일': datetime.datetime(2023, 12, 31, 0, 0),
    '매입부수비용지급일': datetime.datetime(2023, 12, 31, 0, 0)},
   '자금조달일정': {'자기자본유입일': datetime.datetime(2023, 12, 31, 0, 0),
    '차입금유입일': datetime.datetime(2023, 12, 31, 0, 0),
    '이자지급시작일': datetime.datetime(2024, 1, 1, 0, 0),
    '이자지급종료일': datetime.datetime(2026, 12, 31, 0, 0),
    '원금상환시작일': datetime.datetime(2024, 1, 1, 0, 0),
    '원금상환종료일': datetime.datetime(2026, 12, 31, 0, 0)},
   '연간인상률': {'판매단가': 0.05, '운영비': 0.03, '인건비': 0.05}},
  '

In [12]:
cf.dct.keys()

dict_keys(['assumption', 'index', 'funding', 'operating_income', 'operating_cost', 'facility_cost', 'cashflow', 'balance', 'pivot_cashflow', 'print_cashflow'])